In [1]:
import numpy as np
from natsort import natsorted
import scanpy as sc

In [2]:
import numpy as np
import pandas as pd
import os
import glob
import itertools
import scanpy as sc
import natsort

from scroutines import basicu

In [3]:
selected_genes = [
    "Gabra1",
    "Gabra2",
    "Gabra3",
    "Gabra4",
    "Gabra5",
    "Gabra6",
    "Gabrg1",
    "Gabrg2",
    "Gabrg3",
    "Gabrb1",
    "Gabrb2",
    "Gabrb3",
    "Gabrd",    
    "Gabre",    
    "Gabrp",    
    "Gabrq",    
    "Gabrr1",
    "Gabrr2",
    "Gabrr3",
]
len(selected_genes)

19

In [4]:
ddir = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/cheng21_cell_scrna/organized'
outdir = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results/GABAARs'
!mkdir -p $outdir

In [5]:
%%time

f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/All-RNA_v5.h5ad'
adata_all = sc.read(f)

CPU times: user 1.69 s, sys: 22.6 s, total: 24.3 s
Wall time: 3min


In [6]:
unq_samples = natsorted(adata_all.obs['Sample'].unique())
print(len(unq_samples))
print(unq_samples) 

25
['P6a', 'P6b', 'P6c', 'P8a', 'P8b', 'P8c', 'P10a', 'P10b', 'P12DRa', 'P12DRb', 'P12a', 'P12b', 'P12c', 'P14DRa', 'P14DRb', 'P14a', 'P14b', 'P17DRa', 'P17DRb', 'P17a', 'P17b', 'P21DRa', 'P21DRb', 'P21a', 'P21b']


In [7]:
selected_genes_idx = basicu.get_index_from_array(adata_all.var.index.values, selected_genes)
assert np.any(selected_genes_idx != -1)

In [8]:
### read in the file and prep

for i, this_samp in enumerate(unq_samples):
    adata = adata_all[adata_all.obs['Sample']==this_samp]
    
    ### sum over counts 
    # by type
    type_lbls = adata.obs['Subclass'].values
    unq_types = np.unique(type_lbls)

    nc, ng =  len(unq_types), len(adata.var)  #  rep, subclass, gene 
    mat = np.zeros((nc, ng))
    print(mat.shape)

    for k, this_type in enumerate(unq_types):
        selector = (type_lbls==this_type)
        mat[k] = np.sum(np.array(adata[selector].raw.X.todense()), axis=0) # raw reads sum over all cells

    ### CPM  - per sample and subclass
    # normalize it as CPM
    mat = (mat/np.sum(mat, axis=-1, keepdims=True))*1e6

    ### select GABARRs
    submat = mat[:,selected_genes_idx]
    print(submat.shape)

    ### save as csv (one per sample)
    output = os.path.join(outdir, f'GABARRs_Jain_multiome_subclass_CPM_{this_samp}.csv')
    print(this_samp, output)

    submat_df = pd.DataFrame(submat, index=unq_types, columns=selected_genes)
    submat_df.to_csv(output)
    
    # break


(19, 20124)
(19, 19)
P6a /u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results/GABAARs/GABARRs_Jain_multiome_subclass_CPM_P6a.csv
(19, 20124)
(19, 19)
P6b /u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results/GABAARs/GABARRs_Jain_multiome_subclass_CPM_P6b.csv
(20, 20124)
(20, 19)
P6c /u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results/GABAARs/GABARRs_Jain_multiome_subclass_CPM_P6c.csv
(19, 20124)
(19, 19)
P8a /u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results/GABAARs/GABARRs_Jain_multiome_subclass_CPM_P8a.csv
(20, 20124)
(20, 19)
P8b /u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results/GABAARs/GABARRs_Jain_multiome_subclass_CPM_P8b.csv
(20, 20124)
(20, 19)
P8c /u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results/GABAARs/GABARRs_Jain_multiome_subclass_CPM_P8c.csv
(21, 20124)
(21, 19)
P10a /u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results/GABAARs/GABARRs_Jain_multiome_subclass_CPM_P10a.csv
(21, 20124)
(21, 19)
P10b /u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results/GABAARs/GABARRs_

In [9]:
adata.obs.groupby(['Sample', 'Subclass']).size().unstack()

Subclass,Astro,Endo,Frem1,L2/3,L4,L5IT,L5NP,L5PT,L6CT,L6IT,L6b,Lamp5,Micro,OD,OPC,Pvalb,Sst,Stac,VLMC,Vip
Sample,,,,,,,,,,,,,,,,,,,,
P21b,428,59,20,1020,994,189,96,174,655,209,46,74,407,387,256,226,169,15,9,97
